In [1]:
import metrics_calc

Using device: cpu


In [10]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5TokenizerFast, Trainer, TrainingArguments

# Check if GPU (CUDA) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Load trained T5 model and tokenizer
model_path = "./dream_model"
tokenizer = T5TokenizerFast.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

Using device: cpu


In [12]:
# Read validation dataset (Excel file)
input_excel = "dreams_and_interpretations_Freud.xlsx"  # Replace with your actual Excel file name
df = pd.read_excel(input_excel)

output_csv = "dreams_with_generated_interpretations_t5_small.csv"


In [14]:
# Ensure the dataset has a 'Dream' column
if 'Dream' not in df.columns:
    raise ValueError("Excel file must contain a 'Dream' column.")

# # Function to generate interpretations
# def generate_interpretation(dream):
#     input_text = f"interpret the dream: {dream}"
    
#     # Tokenize and move input tensors to the correct device (GPU if available)
#     input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

#     # Generate text
#     output_ids = model.generate(input_ids, max_length=50, num_return_sequences=1)
#     interpretation = tokenizer.decode(output_ids[0], skip_special_tokens=True)

#     return interpretation.strip()

def generate_interpretation(dream):
    input_text = f"interpret the dream: {dream}"
    
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Improved decoding strategy
    output_ids = model.generate(
        input_ids, 
        max_length=50, 
        num_return_sequences=1, 
        temperature=0.7,  # Controls randomness (lower = more deterministic)
        top_k=50,         # Filters unlikely words (smaller = safer)
        top_p=0.92,       # Nucleus sampling
        repetition_penalty=1.2,  # Penalizes repetition
        no_repeat_ngram_size=2   # Avoids repeating 2-grams
    )
    
    interpretation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return interpretation.strip()


# Generate interpretations
df["Generated_Interpretation"] = df["Dream"].apply(generate_interpretation)

# Save to new CSV
df.to_csv(output_csv, index=False)

print(f"Generated interpretations saved to {output_csv}")


C:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.92` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated interpretations saved to dreams_with_generated_interpretations_t5_small.csv


In [15]:
output_metrics_csv = "t5_small_metrics_results.csv"
metrics_calc.evaluate_text_metrics(tokenizer, model, output_csv, output_metrics_csv)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Metrics saved to 't5_small_metrics_results.csv'
